<a href="https://colab.research.google.com/github/LimHyeongSoo/Choong-Ang_Final_Capstone_CVProject_Andrew/blob/main/%08CV_Project.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

## 필요 라이브러리 import

In [ ]:
!pip install tensorflow
!pip install git+https://github.com/ultralytics/yolov5
!pip install ultralytics
!pip install transformers torch torchvision

## 이미지 검출 훈련(자동차 or 버스)

In [ ]:
from ultralytics import YOLO

class YOLOv8Model:
    def __init__(self, model_path=None):
        if model_path is not None:
            self.model = YOLO(weights=model_path)
        else:
            self.model = YOLO()

    def load_dataset(self, dataset_path):
        """데이터셋 로드"""
        self.model.dataset(dataset_path)

    def train(self):
        """모델 훈련 시작"""
        self.model.fit()

    def save_model(self, save_path):
        """훈련된 모델 저장"""
        self.model.save(save_path)

    def detect(self, image_path):
        """이미지에서 객체 검출"""
        results = self.model(image_path)
        results.show()
        return results

# 사용
if __name__ == "__main__":
    # 모델 훈련
    trainer = YOLOv8Model()
    trainer.load_dataset('/content/drive/MyDrive/졸업논문_자율주행자동차/Dataset.yaml')
    trainer.train()
    trainer.save_model('/content/drive/MyDrive/졸업논문_자율주행자동차/model.pt')

    # 훈련된 모델 로드 및 객체 검출
    detector = YOLOv8Model(model_path='/content/drive/MyDrive/졸업논문_자율주행자동차/model.pt')
    detection_results = detector.detect('/content/drive/MyDrive/졸업논문_자율주행자동차/image.jpg')


## 차선 검출 & 방향성(각도 검출)

In [ ]:
import cv2
import numpy as np
import tensorflow as tf

class DeepLabLaneDetector:
    def __init__(self, model_path):
        """모델 초기화 및 로드"""
        self.model = tf.saved_model.load(model_path)

    def predict(self, image_path):
        """이미지에서 차선과 객체를 검출"""
        image = tf.io.read_file(image_path)
        image = tf.image.decode_png(image, channels=3)
        image = tf.image.resize(image, [513, 513])
        image = tf.cast(image, tf.float32) / 127.5 - 1
        batch_image = tf.expand_dims(image, 0)

        # 모델 예측
        output = self.model(batch_image)
        logits = output['logits']
        semantic_prediction = tf.argmax(logits, axis=-1)
        semantic_prediction = tf.squeeze(semantic_prediction).numpy()

        return semantic_prediction, image.numpy()

    def detect_lines(self, image):
        """이미지에서 선을 검출하기 위해 Hough 변환 사용"""
        gray = cv2.cvtColor(image, cv2.COLOR_BGR2GRAY)
        edges = cv2.Canny(gray, 50, 150, apertureSize=3)
        lines = cv2.HoughLinesP(edges, 1, np.pi / 180, threshold=100, minLineLength=100, maxLineGap=10)
        return lines

    def calculate_lane_orientation(self, lines):
        """검출된 선으로부터 차선의 방향성 계산"""
        if lines is not None:
            line = lines[0]  # 대표 선을 사용
            orientation = np.arctan2(line[1] - line[3], line[0] - line[2]) * 180 / np.pi
            return orientation
        return None

    def visualize(self, image, prediction):
        """검출 결과 시각화"""
        colored_prediction = np.zeros_like(image)
        colored_prediction[prediction == 1] = [255, 0, 0]  # 차선을 빨간색으로 표시
        colored_image = cv2.addWeighted(image, 0.5, colored_prediction, 0.5, 0)
        cv2.imshow('Lane Detection', colored_image)
        cv2.waitKey(0)
        cv2.destroyAllWindows()

# 사용 예시
if __name__ == "__main__":
    detector = DeepLabLaneDetector('/content/drive/MyDrive/졸업논문_자율주행자동차/deeplab (1)/model')
    prediction, original_image = detector.predict('/content/drive/MyDrive/졸업논문_자율주행자동차/deeplab (1)/image.png')
    lines_detected = detector.detect_lines((original_image * 127.5 + 127.5).astype(np.uint8))
    lane_orientation = detector.calculate_lane_orientation(lines_detected)
    detector.visualize(original_image, prediction)
    print(f"Lane Orientation: {lane_orientation} degrees")


## 도로 구조물 종류 검출(우선 clip을 이용하여 도로 구조물 분류 작업)

In [ ]:
from transformers import CLIPProcessor, CLIPModel
import torch
from PIL import Image
import requests
from io import BytesIO

class CLIPRoadStructureDetector:
    def __init__(self, model_name="openai/clip-vit-base-patch32"):
        """모델과 프로세서 초기화"""
        self.model = CLIPModel.from_pretrained(model_name)
        self.processor = CLIPProcessor.from_pretrained(model_name)

    def load_prompts(self, file_path):
        """프롬프트 파일 로드"""
        with open(file_path, 'r') as file:
            prompts = [line.strip() for line in file if line.strip()]
        return prompts

    def detect_structure(self, image_url, prompts_file):
        """이미지에서 도로 구조물 검출"""
        # 이미지 로드
        response = requests.get(image_url)
        pil_image = Image.open(BytesIO(response.content)).convert("RGB")

        # 프롬프트 로드
        prompts = self.load_prompts(prompts_file)

        # 입력 처리 및 모델 실행
        inputs = self.processor(text=prompts, images=pil_image, return_tensors="pt", padding=True)
        outputs = self.model(**inputs)
        logits_per_image = outputs.logits_per_image
        probs = logits_per_image.softmax(dim=1)

        # 가장 높은 점수와 해당 프롬프트 출력
        max_index = probs.argmax()
        best_prompt = prompts[max_index]
        best_score = probs[0, max_index].item()  # 확률 점수

        return best_prompt, best_score

# 사용 예시
if __name__ == "__main__":
    detector = CLIPRoadStructureDetector()
    best_match, score = detector.detect_structure("/content/drive/MyDrive/졸업논문_자율주행자동차/structure/test.jpg", "/content/drive/MyDrive/졸업논문_자율주행자동차/structure/prompts.txt")
    print(f"Best match: {best_match} with a score of {score:.4f}")
